In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display

%matplotlib inline

# 5 モデルの評価と改良

from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = make_blobs(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
logreg = LogisticRegression().fit(X_train, y_train)
print("Test set score: {:.2f}".format(logreg.score(X_test, y_test)))

Test set score: 0.88


In [15]:
# 5.1 交差検証 cross-validation
#
# 汎化性能を評価
# 最もよく用いられる -> k分割交差検証 k-fold cross-validation
# k: 5~10 (in most case)
#
# Use sklearn.model_selection.cross_val_score()

from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

iris = load_iris()
logreg = LogisticRegression()

scores = cross_val_score(logreg, iris.data, iris.target)
print(f"Cross-validation scores: {scores}")

Cross-validation scores: [ 0.96078431  0.92156863  0.95833333]


In [16]:
# 5 folds
scores = cross_val_score(logreg, iris.data, iris.target, cv=5)
print(f"Cross-validation scores: {scores}")

Cross-validation scores: [ 1.          0.96666667  0.93333333  0.9         1.        ]


In [17]:
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Average cross-validation score: 0.96


In [18]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)

print(f"Cross-validation scores:\n{cross_val_score(logreg, iris.data, iris.target, cv=kfold)}")

Cross-validation scores:
[ 1.          0.93333333  0.43333333  0.96666667  0.43333333]


In [19]:
kfold = KFold(n_splits=3)

print(f"Cross-validation scores:\n{cross_val_score(logreg, iris.data, iris.target, cv=kfold)}")

Cross-validation scores:
[ 0.  0.  0.]


In [20]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)

print(f"Cross-validation scores:\n{cross_val_score(logreg, iris.data, iris.target, cv=kfold)}")

Cross-validation scores:
[ 0.9   0.96  0.96]


In [21]:
# 1つ抜き交差検証 leave-one-out

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print("Number of cv iterations: ", len(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))

Number of cv iterations:  150
Mean accuracy: 0.95


In [22]:
# シャッフル分割交差検証 shuffle-split cross-validation

from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)
print(f"Cross-validation scores:\n{scores}")

Cross-validation scores:
[ 0.90666667  0.96        0.92        0.92        0.96        0.92
  0.97333333  0.84        0.76        0.94666667]


In [23]:
# グループ付き交差検証

from sklearn.model_selection import GroupKFold

X, y = make_blobs(n_samples=12, random_state=0)
groups = [0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3]
scores = cross_val_score(logreg, X, y, groups, cv=GroupKFold(n_splits=3))
print(f"Cross-validation scores:\n{scores}")

Cross-validation scores:
[ 0.75        0.8         0.66666667]


In [24]:
# 5.2 グリッドサーチ

from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
print(f"Size of training set: {X_train.shape[0]} size of test set: {X_test.shape[0]}")

best_score = 0

arr = [0.001, 0.01, 0.1, 1, 10, 100] 
for gamma in arr:
    for C in arr:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        score = svm.score(X_test, y_test)
        
        if score > best_score:
            best_score = score
            best_params = {'C': C, 'gamma': gamma}
            
print("Best score: {:.2f}".format(best_score))
print("Best params: {}".format(best_params))

Size of training set: 112 size of test set: 38
Best score: 0.97
Best params: {'C': 100, 'gamma': 0.001}


In [26]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    iris.data, iris.target, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_trainval, y_trainval, random_state=1)
print(f"Size of training set: {X_train.shape[0]}  size of validation set: {X_valid.shape[0]} size of test set: {X_test.shape[0]}")

best_score = 0

arr = [0.001, 0.01, 0.1, 1, 10, 100] 
for gamma in arr:
    for C in arr:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        score = svm.score(X_valid, y_valid)
        
        if score > best_score:
            best_score = score
            best_params = {'C': C, 'gamma': gamma}
            
svm = SVC(**best_params)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)
print("Best score on validation set: {:.2f}".format(best_score))
print("Best params: {}".format(best_params))
print("Test set score with best params: {:.2f}".format(test_score))

Size of training set: 84  size of validation set: 28 size of test set: 38
Best score on validation set: 0.96
Best params: {'C': 10, 'gamma': 0.001}
Test set score with best params: 0.92


In [27]:
arr = [0.001, 0.01, 0.1, 1, 10, 100] 
param_grid = {'C': arr, 'gamma': arr}

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
grid_search = GridSearchCV(SVC(), param_grid, cv=5)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
grid_search.fit(X_train, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

Test set score: 0.97


In [30]:
print(f"Best params: {grid_search.best_params_}")
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

Best params: {'C': 100, 'gamma': 0.01}
Best cross-validation score: 0.97


In [31]:
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Best estimator:
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [32]:
results = pd.DataFrame(grid_search.cv_results_)
display(results.head())

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:1

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_gamma,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.002713,0.000997,0.366071,0.366079,0.001,0.001,"{'C': 0.001, 'gamma': 0.001}",22,0.375,0.363636,...,0.363636,0.366667,0.363636,0.366667,0.380952,0.362637,0.001220,0.000387,0.011371,0.002852
1,0.001661,0.000618,0.366071,0.366079,0.001,0.01,"{'C': 0.001, 'gamma': 0.01}",22,0.375,0.363636,...,0.363636,0.366667,0.363636,0.366667,0.380952,0.362637,0.000229,0.000070,0.011371,0.002852
2,0.001446,0.000599,0.366071,0.366079,0.001,0.1,"{'C': 0.001, 'gamma': 0.1}",22,0.375,0.363636,...,0.363636,0.366667,0.363636,0.366667,0.380952,0.362637,0.000082,0.000047,0.011371,0.002852
3,0.001419,0.000576,0.366071,0.366079,0.001,1,"{'C': 0.001, 'gamma': 1}",22,0.375,0.363636,...,0.363636,0.366667,0.363636,0.366667,0.380952,0.362637,0.000135,0.000025,0.011371,0.002852
4,0.001460,0.000515,0.366071,0.366079,0.001,10,"{'C': 0.001, 'gamma': 10}",22,0.375,0.363636,...,0.363636,0.366667,0.363636,0.366667,0.380952,0.362637,0.000209,0.000088,0.011371,0.002852
